In [1]:
# Import Libraries

import pandas as pd
import requests as req
import json
from datetime import datetime, timedelta
import numpy as np

pd.options.display.max_rows = 400

In [2]:
# Retrieve RW Metadata

# Base URL for getting dataset metadata from RW API
# Metadata = Data that describes Data
url = "https://api.resourcewatch.org/v1/dataset?sort=slug,-provider,userId&status=saved&includes=metadata,vocabulary,widget,layer"

# Page[size] tells the API the maximum number of results to send back
# There are currently between 200 and 300 datasets on the RW API

payload = { 'application':'rw', 'page[size]': 1000}

# Request all datasets, and extract the data from the response
res = req.get(url, params=payload)
data = res.json()['data']

#######################################################

In [3]:
### Convert the json object returned by the API into a pandas DataFrame
# Another option: https://pandas.pydata.org/pandas-docs/stable/generated/pandas.io.json.json_normalize.html

datasets_on_api = {}
for ix, dset in enumerate(data):
    atts = dset['attributes']
    metadata = atts['metadata']
    layers = atts['layer']
    widgets = atts['widget']
    tags = atts['vocabulary']
    datasets_on_api[atts['name']] = {
        'rw_id': dset['id'],
        'table_name': atts['tableName'],
        'provider': atts['provider'],
        'date_updated': atts['updatedAt'],
        'num_metadata': len(metadata),
        'metadata': metadata,
        'num_layers': len(layers),
        'layers': layers,
        'num_widgets': len(widgets),
        'widgets': widgets,
        'num_tags': len(tags),
        'tags': tags,
    }   

In [4]:
# Create the DataFrame, name the index, and sort by date_updated
# More recently updated datasets at the top

current_datasets_on_api = pd.DataFrame.from_dict(datasets_on_api, orient='index')
current_datasets_on_api.index.rename('Dataset', inplace=True)
current_datasets_on_api.sort_values(by=['date_updated'], inplace=True, ascending=False)

In [6]:
# Import, Format, Export Data

refugees_csv = pd.read_excel('CSVs/Refugees_CSV.xlsx').drop(['Year', 'Refugee'], axis=1)
refugees = refugees_csv.groupby('Country').sum().reset_index()
refugees = refugees.set_index('Country')
refugees['Refugees'] = refugees['Refugees'].astype(np.float64)

print(type(refugees['Refugees'][0]))
refugees['Refugees'].max()

<class 'numpy.float64'>


6308619.0

In [7]:
## Connecting refugee data with wri_bounds

bounds = req.get('https://raw.githubusercontent.com/wri/wri-bounds/master/dist/all_primary_countries.geojson').json()

In [8]:
refugees

Refugees
Country                                             
Afghanistan                                2624225.0
Albania                                      12163.0
Algeria                                       3991.0
American Samoa                                   0.0
Andorra                                          0.0
Angola                                        8267.0
Antigua and Barbuda                             94.0
Argentina                                      111.0
Armenia                                      10766.0
Australia                                        0.0
Austria                                          0.0
Azerbaijan                                   10878.0
Bahamas                                        373.0
Bahrain                                        487.0
Bangladesh                                   16755.0
Barbados                                       167.0
Belarus                                       3635.0
Belgium                                         52.0
Belize                                          59.0
Benin                                          557.0
Bhutan                                        7929.0
Bolivia (Plurinational State of)               505.0
Bosnia and Herzegovina                       17669.0
Botswana                                       279.0
Brazil                                         836.0
Brunei Darussalam                                0.0
Bulgaria                                       716.0
Burkina Faso                                  2679.0
Burundi                                     439280.0
Cabo Verde                                       0.0
Cambodia                                     12213.0
Cameroon                                     10982.0
Canada                                          75.0
Cayman Islands                                   6.0
Central African Rep.                        545498.0
Chad                                         16295.0
Chile                                          483.0
China                                       207691.0
China, Hong Kong SAR                             6.0
China, Macao SAR                                 0.0
Colombia                                    191607.0
Comoros                                        575.0
Congo                                        13225.0
Cook Islands                                     0.0
Costa Rica                                     190.0
Cote d'Ivoire                                39939.0
Croatia                                      24859.0
Cuba                                          5316.0
CuraÃ§ao                                        35.0
Cyprus                                           0.0
Czech Rep.                                    1272.0
Dem. People's Rep. of Korea                   1161.0
Dem. Rep. of the Congo                      620775.0
Denmark                                          0.0
Djibouti                                      1741.0
Dominica                                        26.0
Dominican Rep.                                 412.0
Ecuador                                       1236.0
Egypt                                        22056.0
El Salvador                                  25861.0
Equatorial Guinea                              122.0
Eritrea                                     486169.0
Estonia                                        278.0
Ethiopia                                     87388.0
Federated States of Micronesia                   0.0
Fiji                                           734.0
Finland                                          0.0
France                                          49.0
French Guiana                                    0.0
Gabon                                          211.0
Gambia                                       14539.0
Georgia                                       6485.0
Germany                                         66.0
Ghana                                        16929.0
Gibraltar       

In [10]:
for ix, cntry in enumerate(bounds['features']):
    name = cntry['properties']['name']
    if name not in refugees.index:
        print(name)
        new_name = input('What should new name be?')
        if new_name:
            bounds['features'][ix]['properties']['name'] = new_name
            bounds['features'][ix]['properties']['Refugees'] = refugees.loc[new_name, 'Refugees']
        else:
            print('no data')
            bounds['features'][ix]['properties']['Refugees'] = None
    else:
        bounds['features'][ix]['properties']['Refugees'] = refugees.loc[name, 'Refugees']

Dem. Rep of the Congo
What should new name be?Dem. Rep. of the Congo
Republic of Congo
What should new name be?Congo
Cape Verde
What should new name be?Cabo Verde
Czech Republic
What should new name be?Czech Rep.
Dominican Republic
What should new name be?Dominican Rep.
Guinea Bissau
What should new name be?Guinea-Bissau
Iran
What should new name be?Iran (Islamic Rep. of)
South Korea
What should new name be?Rep. of Korea
Laos
What should new name be?Lao People's Dem. Rep.
Liechtenstein
What should new name be?Lichtenstein
Moldova
What should new name be?Rep. of Moldova
Macedonia
What should new name be?The former Yugoslav Republic of Macedonia
Nauru
What should new name be?
no data
North Korea
What should new name be?Dem. People's Rep. of Korea
Russia
What should new name be?Russian Federation
Republic of Serbia
What should new name be?Serbia and Kosovo (S/RES/1244 (1999))
Syria
What should new name be?Syrian Arab Rep.
United Republic of Tanzania
What should new name be?United Rep. of 

In [11]:
json.dump(bounds, open('bounds_with_refugee_counts.geojson', 'w'))

In [12]:
## Creating data frame for scatter plots
refugees = refugees.reset_index()
refugees

Country   Refugees
0                                  Afghanistan  2624225.0
1                                      Albania    12163.0
2                                      Algeria     3991.0
3                               American Samoa        0.0
4                                      Andorra        0.0
5                                       Angola     8267.0
6                          Antigua and Barbuda       94.0
7                                    Argentina      111.0
8                                      Armenia    10766.0
9                                    Australia        0.0
10                                     Austria        0.0
11                                  Azerbaijan    10878.0
12                                     Bahamas      373.0
13                                     Bahrain      487.0
14                                  Bangladesh    16755.0
15                                    Barbados      167.0
16                                     Belarus     3635.0
17                                     Belgium       52.0
18                                      Belize       59.0
19                                       Benin      557.0
20                                      Bhutan     7929.0
21            Bolivia (Plurinational State of)      505.0
22                      Bosnia and Herzegovina    17669.0
23                                    Botswana      279.0
24                                      Brazil      836.0
25                           Brunei Darussalam        0.0
26                                    Bulgaria      716.0
27                                Burkina Faso     2679.0
28                                     Burundi   439280.0
29                                  Cabo Verde        0.0
30                                    Cambodia    12213.0
31                                    Cameroon    10982.0
32                                      Canada       75.0
33                              Cayman Islands        6.0
34                        Central African Rep.   545498.0
35                                        Chad    16295.0
36                                       Chile      483.0
37                                       China   207691.0
38                        China, Hong Kong SAR        6.0
39                            China, Macao SAR        0.0
40                                    Colombia   191607.0
41                                     Comoros      575.0
42                                       Congo    13225.0
43                                Cook Islands        0.0
44                                  Costa Rica      190.0
45                               Cote d'Ivoire    39939.0
46                                     Croatia    24859.0
47                                        Cuba     5316.0
48                                    CuraÃ§ao       35.0
49                                      Cyprus        0.0
50                                  Czech Rep.     1272.0
51                 Dem. People's Rep. of Korea     1161.0
52                      Dem. Rep. of the Congo   620775.0
53                                     Denmark        0.0
54                                    Djibouti     1741.0
55                                    Dominica       26.0
56                              Dominican Rep.      412.0
57                                     Ecuador     1236.0
58                                       Egypt    22056.0
59                                 El Salvador    25861.0
60                           Equatorial Guinea      122.0
61                                     Eritrea   486169.0
62                                     Estonia      278.0
63                                    Ethiopia    87388.0
64              Federated States of Micronesia        0.0
65                                        Fiji      734.0
66                                     Finland        0.0
67                                      France       49.0
68                               Fren

In [15]:
### Water Stress
query_base = "https://api.resourcewatch.org/v1/query/{}?sql={}"

sql = "SELECT * FROM {}"
DT_FORMAT = '%Y-%m-%dT%H:%M:%SZ'

today = datetime.today()
five_years_ago = today - timedelta(days=360*5)

#[ds for ds in current_datasets_on_api.index if 'stress' in ds.lower()]

ds = [ds for ds in current_datasets_on_api.index if 'stress' in ds.lower()][0]
ds_id = current_datasets_on_api.loc[ds, 'rw_id']
table_name = current_datasets_on_api.loc[ds, 'table_name']

#query_sql = sql.format(table_name, cutoff)
query_sql = sql.format(table_name)
query = query_base.format(ds_id, query_sql)
res = req.get(query).json()
water_stress = pd.DataFrame(res['data'])

In [16]:
water_stress = water_stress[['all_sectors', 'name']]

In [17]:
### GDP 
query_base = "https://api.resourcewatch.org/v1/query/{}?sql={}"

sql = "SELECT * FROM {} WHERE year = 2015"
DT_FORMAT = '%Y-%m-%dT%H:%M:%SZ'

#[ds for ds in current_datasets_on_api.index if 'gross' in ds.lower()]

ds = [ds for ds in current_datasets_on_api.index if 'gross' in ds.lower()][0]
ds_id = current_datasets_on_api.loc[ds, 'rw_id']
table_name = current_datasets_on_api.loc[ds, 'table_name']

query_sql = sql.format(table_name)
query = query_base.format(ds_id, query_sql)
res = req.get(query).json()

gdp = pd.DataFrame(res['data'])

In [18]:
gdp = gdp[['rw_country_code', 'rw_country_name', 'yr_data']]

In [19]:
### Political Freedoms Index
query_base = "https://api.resourcewatch.org/v1/query/{}?sql={}"

sql = "SELECT * FROM {}"
DT_FORMAT = '%Y-%m-%dT%H:%M:%SZ'

#[ds for ds in current_datasets_on_api.index if 'political' in ds.lower()]

ds = [ds for ds in current_datasets_on_api.index if 'political' in ds.lower()][1]
ds_id = current_datasets_on_api.loc[ds, 'rw_id']
table_name = current_datasets_on_api.loc[ds, 'table_name']

query_sql = sql.format(table_name)
query = query_base.format(ds_id, query_sql)
res = req.get(query).json()

political_freedoms = pd.DataFrame(res['data'])

In [20]:
political_freedoms = political_freedoms[['rw_country_code', 'rw_country_name', "total_aggr"]]

In [21]:
## Joined

tmp = political_freedoms.merge(gdp, left_on='rw_country_code', right_on='rw_country_code')

In [22]:
aliases = tmp[['rw_country_code', 'rw_country_name_x']]
aliases.columns = ['code', 'name']

In [24]:
water_stress

all_sectors                              name
0       5.000000               Antigua and Barbuda
1       5.000000                           Bahrain
2       5.000000                          Barbados
3       5.000000                           Comoros
4       5.000000                            Cyprus
5       5.000000                          Dominica
6       5.000000                           Jamaica
7       5.000000                             Malta
8       5.000000                             Qatar
9       5.000000                       Saint Lucia
10      5.000000  Saint Vincent and the Grenadines
11      5.000000                        San Marino
12      5.000000                         Singapore
13      5.000000               Trinidad and Tobago
14      5.000000              United Arab Emirates
15      5.000000                    Western Sahara
16      4.992351                      Saudi Arabia
17      4.964874                            Kuwait
18      4.913786                              Oman
19      4.840212                             Libya
20      4.825386                            Israel
21      4.821998                        Kyrgyzstan
22      4.806092                        East Timor
23      4.781554                              Iran
24      4.674046                             Yemen
25      4.633905                         Palestine
26      4.594450                            Jordan
27      4.540997                           Lebanon
28      4.379483                        Somaliland
29      4.316672                        Uzbekistan
30      4.310025                          Pakistan
31      4.299610                      Turkmenistan
32      4.236047                           Morocco
33      4.046934                          Mongolia
34      4.018548                        Kazakhstan
35      4.009098                       Afghanistan
36      3.972669                           Lesotho
37      3.851933                             Syria
38      3.725614                             Spain
39      3.584887                             India
40      3.543805                       South Korea
41      3.528026                        Tajikistan
42      3.516054                            Mexico
43      3.511488                         Australia
44      3.487299                Dominican Republic
45      3.483185                              Iraq
46      3.444267                           Algeria
47      3.441953                           Tunisia
48      3.399435                           Vatican
49      3.391520                        Azerbaijan
50      3.390809                          Djibouti
51      3.350268                             Italy
52      3.338300                          Portugal
53      3.333262                       Philippines
54      3.326591                           Andorra
55      3.272571                            Greece
56      3.264861                         Indonesia
57      3.205366                             Chile
58      3.201609                              Peru
59      3.192193                              Cuba
60      3.164518                           Belgium
61      3.105144                         Swaziland
62      3.067363                           Armenia
63      3.054322                             Japan
64      3.040457                      South Africa
65      3.020970                            Turkey
66      3.018732                           Eritrea
67      3.007875                         Sri Lanka
68      2.944769                             China
69      2.917040                           Ireland
70      2.888310          United States of America
71      2.750275                           Estonia
72      2.664468                            Monaco
73      2.645752                         Macedonia
74      2.628115                    United Kingdom
75      2.513095                         Argentina
76      2.508314                        Luxembourg
77      2.402084           

In [26]:
def find_rw_country_code(name, aliases):
    if name in aliases['name'].values:
        code = aliases.loc[aliases['name'].isin([name]), 'code'].values[0]
        return code
    else:
        print(name)
        code = input('Country Code?')
        if code:
            return code
        else:
            return None
        
water_stress['rw_country_code'] = water_stress.apply(lambda row: find_rw_country_code(row['name'], aliases), axis = 1)
water_stress

Western Sahara
Country Code?ESH
East Timor
Country Code?TLS
Palestine
Country Code?PSE
Somaliland
Country Code?SOM
Vatican
Country Code?VAT
Kosovo
Country Code?SRB
Republic of the Congo
Country Code?COG
Taiwan
Country Code?TWN
Curacao
Country Code?CUW


all_sectors                              name rw_country_code
0       5.000000               Antigua and Barbuda             ATG
1       5.000000                           Bahrain             BHR
2       5.000000                          Barbados             BRB
3       5.000000                           Comoros             COM
4       5.000000                            Cyprus             CYP
5       5.000000                          Dominica             DMA
6       5.000000                           Jamaica             JAM
7       5.000000                             Malta             MLT
8       5.000000                             Qatar             QAT
9       5.000000                       Saint Lucia             LCA
10      5.000000  Saint Vincent and the Grenadines             VCT
11      5.000000                        San Marino             SMR
12      5.000000                         Singapore             SGP
13      5.000000               Trinidad and Tobago             TTO
14      5.000000              United Arab Emirates             ARE
15      5.000000                    Western Sahara             ESH
16      4.992351                      Saudi Arabia             SAU
17      4.964874                            Kuwait             KWT
18      4.913786                              Oman             OMN
19      4.840212                             Libya             LBY
20      4.825386                            Israel             ISR
21      4.821998                        Kyrgyzstan             KGZ
22      4.806092                        East Timor             TLS
23      4.781554                              Iran             IRN
24      4.674046                             Yemen             YEM
25      4.633905                         Palestine             PSE
26      4.594450                            Jordan             JOR
27      4.540997                           Lebanon             LBN
28      4.379483                        Somaliland             SOM
29      4.316672                        Uzbekistan             UZB
30      4.310025                          Pakistan             PAK
31      4.299610                      Turkmenistan             TKM
32      4.236047                           Morocco             MAR
33      4.046934                          Mongolia             MNG
34      4.018548                        Kazakhstan             KAZ
35      4.009098                       Afghanistan             AFG
36      3.972669                           Lesotho             LSO
37      3.851933                             Syria             SYR
38      3.725614                             Spain             ESP
39      3.584887                             India             IND
40      3.543805                       South Korea             KOR
41      3.528026                        Tajikistan             TJK
42      3.516054                            Mexico             MEX
43      3.511488                         Australia             AUS
44      3.487299                Dominican Republic             DOM
45      3.483185                              Iraq             IRQ
46      3.444267                           Algeria             DZA
47      3.441953                           Tunisia             TUN
48      3.399435                           Vatican             VAT
49      3.391520                        Azerbaijan             AZE
50      3.390809                          Djibouti             DJI
51      3.350268                             Italy             ITA
52      3.338300                          Portugal             PRT
53      3.333262                       Philippines             PHL
54      3.326591                           Andorra             AND
55      3.272571                            Greece             GRC
56      3.264861                         Indonesia             IDN
57      3.205366                             Chile             CHL
58      3.201609                              Peru  

In [27]:
tmp = tmp.merge(water_stress, left_on='rw_country_code', right_on='rw_country_code')
tmp

rw_country_code                 rw_country_name_x  total_aggr  \
0               THA                          Thailand          32   
1               TJK                        Tajikistan          11   
2               ARG                         Argentina          82   
3               MRT                        Mauritania          30   
4               MCO                            Monaco          84   
5               ZWE                          Zimbabwe          32   
6               YEM                             Yemen          14   
7               BHR                           Bahrain          12   
8               VNM                           Vietnam          20   
9               VEN                         Venezuela          30   
10              CRI                        Costa Rica          91   
11              COD  Democratic Republic of the Congo          19   
12              COG                 Republic of Congo          27   
13              CZE                    Czech Republic          94   
14              CIV                       Ivory Coast          52   
15              BRB                          Barbados          97   
16              STP             Sao Tome and Principe          81   
17              SWE                            Sweden         100   
18              MUS                         Mauritius          89   
19              TUV                            Tuvalu          94   
20              UGA                            Uganda          35   
21              VUT                           Vanuatu          80   
22              UKR                           Ukraine          61   
23              TKM                      Turkmenistan           4   
24              UZB                        Uzbekistan           3   
25              PAK                          Pakistan          43   
26              SRB                Republic of Serbia          76   
27              SRB                Republic of Serbia          76   
28              SLB                   Solomon Islands          71   
29              ESP                             Spain          94   
30              SUR                          Suriname          77   
31              KOR                       South Korea          82   
32              IRN                              Iran          17   
33              AUT                           Austria          95   
34              LIE                     Liechtenstein          91   
35              LAO                              Laos          12   
36              LBR                           Liberia          62   
37              KGZ                        Kyrgyzstan          37   
38              MWI                            Malawi          63   
39              SWZ                         Swaziland          18   
40              SWZ                         Swaziland          18   
41              PAN                            Panama          83   
42              PRY                          Paraguay          64   
43              PHL                       Philippines          63   
44              RUS                            Russia          20   
45              SAU                      Saudi Arabia          10   
46              ROU                           Romania          84   
47              PRT                          Portugal          97   
48              PER                              Peru          72   
49              SSD                       South Sudan           4   
50              CHE                       Switzerland          96   
51              SMR                        San Marino          97   
52              GAB                             Gabon          32   
53              DEU                           Germany          95   
54              GRC                            Greece          84   
55              NOR                            Norway         100   
56              NER                             Niger          49   
57

In [29]:
refugees['rw_country_code'] = refugees.apply(lambda row: find_rw_country_code(row['Country'], aliases), axis=1)

American Samoa
Country Code?ASM
Bahamas
Country Code?BHS
Bolivia (Plurinational State of)
Country Code?BOL
Brunei Darussalam
Country Code?BRN
Cabo Verde
Country Code?CPV
Cayman Islands
Country Code?CYM
Central African Rep.
Country Code?CAF
China, Hong Kong SAR
Country Code?HKG
China, Macao SAR
Country Code?MAC
Congo
Country Code?COG
Cook Islands
Country Code?COK
Cote d'Ivoire
Country Code?CIV
CuraÃ§ao
Country Code?CUW
Czech Rep.
Country Code?CZE
Dem. People's Rep. of Korea
Country Code?PRK
Dem. Rep. of the Congo
Country Code?COD
Dominican Rep.
Country Code?DOM
French Guiana
Country Code?GUF
Gibraltar
Country Code?GIB
Guadeloupe
Country Code?GLP
Guam
Country Code?GUM
Guinea-Bissau
Country Code?GNB
Iran (Islamic Rep. of)
Country Code?IRN
Lao People's Dem. Rep.
Country Code?LAO
Lichtenstein
Country Code?LIE
Niue
Country Code?NIU
Palestinian
Country Code?PSE
Puerto Rico
Country Code?PRI
Rep. of Korea
Country Code?KOR
Rep. of Moldova
Country Code?MDA
Russian Federation
Country Code?RUS
Sain

In [30]:
tmp = tmp.merge(refugees, left_on='rw_country_code', right_on='rw_country_code')

In [31]:
tmp = tmp.drop(['rw_country_name_y', 'name', 'Country'], axis=1)

In [32]:
tmp

rw_country_code                 rw_country_name_x  total_aggr  \
0               THA                          Thailand          32   
1               TJK                        Tajikistan          11   
2               ARG                         Argentina          82   
3               MRT                        Mauritania          30   
4               MCO                            Monaco          84   
5               ZWE                          Zimbabwe          32   
6               YEM                             Yemen          14   
7               BHR                           Bahrain          12   
8               VNM                           Vietnam          20   
9               VEN                         Venezuela          30   
10              CRI                        Costa Rica          91   
11              COD  Democratic Republic of the Congo          19   
12              COG                 Republic of Congo          27   
13              CZE                    Czech Republic          94   
14              CIV                       Ivory Coast          52   
15              BRB                          Barbados          97   
16              STP             Sao Tome and Principe          81   
17              SWE                            Sweden         100   
18              MUS                         Mauritius          89   
19              TUV                            Tuvalu          94   
20              UGA                            Uganda          35   
21              VUT                           Vanuatu          80   
22              UKR                           Ukraine          61   
23              TKM                      Turkmenistan           4   
24              UZB                        Uzbekistan           3   
25              PAK                          Pakistan          43   
26              SRB                Republic of Serbia          76   
27              SRB                Republic of Serbia          76   
28              SLB                   Solomon Islands          71   
29              ESP                             Spain          94   
30              SUR                          Suriname          77   
31              KOR                       South Korea          82   
32              IRN                              Iran          17   
33              AUT                           Austria          95   
34              LIE                     Liechtenstein          91   
35              LAO                              Laos          12   
36              LBR                           Liberia          62   
37              KGZ                        Kyrgyzstan          37   
38              MWI                            Malawi          63   
39              SWZ                         Swaziland          18   
40              SWZ                         Swaziland          18   
41              PAN                            Panama          83   
42              PRY                          Paraguay          64   
43              PHL                       Philippines          63   
44              RUS                            Russia          20   
45              SAU                      Saudi Arabia          10   
46              ROU                           Romania          84   
47              PRT                          Portugal          97   
48              PER                              Peru          72   
49              SSD                       South Sudan           4   
50              CHE                       Switzerland          96   
51              SMR                        San Marino          97   
52              GAB                             Gabon          32   
53              DEU                           Germany          95   
54              GRC                            Greece          84   
55              NOR                            Norway         100   
56              NER                             Niger          49   
57

In [33]:
final_data = tmp
final_data.columns = ['Country Code', 'Country Name', 'Political Freedoms Index', 'GDP', 'Water Stress, All Sectors', 'Number of Refugees from each Country']
final_data = pd.melt(final_data, id_vars = ['Country Code', 'Country Name'], value_vars = ['Political Freedoms Index', 'GDP', 'Water Stress, All Sectors', 'Number of Refugees from each Country'])
final_data.to_csv('scatterplotdata.csv')

# PD melt stacked all the values columns on top of one another instead 
# of having them as four separate columns.

In [34]:
final_data

Country Code                      Country Name  \
0            THA                          Thailand   
1            TJK                        Tajikistan   
2            ARG                         Argentina   
3            MRT                        Mauritania   
4            MCO                            Monaco   
5            ZWE                          Zimbabwe   
6            YEM                             Yemen   
7            BHR                           Bahrain   
8            VNM                           Vietnam   
9            VEN                         Venezuela   
10           CRI                        Costa Rica   
11           COD  Democratic Republic of the Congo   
12           COG                 Republic of Congo   
13           CZE                    Czech Republic   
14           CIV                       Ivory Coast   
15           BRB                          Barbados   
16           STP             Sao Tome and Principe   
17           SWE                            Sweden   
18           MUS                         Mauritius   
19           TUV                            Tuvalu   
20           UGA                            Uganda   
21           VUT                           Vanuatu   
22           UKR                           Ukraine   
23           TKM                      Turkmenistan   
24           UZB                        Uzbekistan   
25           PAK                          Pakistan   
26           SRB                Republic of Serbia   
27           SRB                Republic of Serbia   
28           SLB                   Solomon Islands   
29           ESP                             Spain   
30           SUR                          Suriname   
31           KOR                       South Korea   
32           IRN                              Iran   
33           AUT                           Austria   
34           LIE                     Liechtenstein   
35           LAO                              Laos   
36           LBR                           Liberia   
37           KGZ                        Kyrgyzstan   
38           MWI                            Malawi   
39           SWZ                         Swaziland   
40           SWZ                         Swaziland   
41           PAN                            Panama   
42           PRY                          Paraguay   
43           PHL                       Philippines   
44           RUS                            Russia   
45           SAU                      Saudi Arabia   
46           ROU                           Romania   
47           PRT                          Portugal   
48           PER                              Peru   
49           SSD                       South Sudan   
50           CHE                       Switzerland   
51           SMR                        San Marino   
52           GAB                             Gabon   
53           DEU                           Germany   
54           GRC                            Greece   
55           NOR                            Norway   
56           NER                             Niger   
57           SEN                           Senegal   
58           ALB                           Albania   
59           DZA                           Algeria   
60           MLT                             Malta   
61           MLI                              Mali   
62           MDG                        Madagascar   
63           MYS                          Malaysia   
64           MDV                          Maldives   
65           MAR                           Morocco   
66           MEX                            Mexico   
67           NAM                           Namibia   
68           MNG                          Mongolia   
69           MMR                           Myanmar   
70           PRK                       North Korea   
71           NPL                             Nepal   
72           OMN                              Oman   
73      